<a href="https://colab.research.google.com/github/akshataupadhye/SWB-GVCEH-Phase3/blob/main/sentiment_analysis/RedditSentimentAnalysisTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
from transformers import pipeline

In [ ]:
def sentiment_analysis_reddit(text):
    # Initialize sentiment analysis pipeline
    sentiment_analyzer = pipeline('sentiment-analysis', model="cardiffnlp/twitter-roberta-base-sentiment-latest", device=-1, truncation=True)
    sentiment_analyzer.tokenizer.model_max_length = 512

    # Analyze sentiment for each segment
    sentiment_results = []

    # Counters for sentiments and scores
    sentiment_counts = {'positive': 0, 'negative': 0, 'neutral': 0}
    sentiment_scores = {'positive': 0, 'negative': 0, 'neutral': 0}

    # Segment the text
    segments = text.split('\n')  # Split by newline

    for segment in segments:
        # Optionally, filter out segments that are too short or not meaningful
        if len(segment.strip()) > 0:
            result = sentiment_analyzer(segment)[0]
            sentiment_label = result['label']
            sentiment_score = result['score']

            sentiment_results.append({
                'segment': segment,
                'sentiment_label': sentiment_label,
                'scores': sentiment_score
            })

            # Update sentiment and score counters
            sentiment_counts[sentiment_label] += 1
            sentiment_scores[sentiment_label] += sentiment_score

  # Determine overall sentiment based on total scores
    if sum(sentiment_counts.values()) > 0:  # Check if any segment with non-zero sentiment scores
        overall_sentiment = max(sentiment_scores, key=sentiment_scores.get)
        overall_sentiment_score = sentiment_scores[overall_sentiment] / sentiment_counts[overall_sentiment]
    else:
        overall_sentiment = None
        overall_sentiment_score = None


    return {'label': overall_sentiment.capitalize(), 'score': overall_sentiment_score}


In [ ]:
test_df = pd.read_csv('/content/test_set_sentiment.csv', index=False)

In [ ]:
test_df.head()

,Unnamed: 0,index,Subreddit,Title,Text,TitleText,manual_sentiment_label,Sentiment_Full,Sentence_Level_Sentiment_Compund,Relevent_Sentence_Sentiment_Compund,sentiment_TextBlob_Full,sentiment_TextBlob_Compound,relevant_sent_sentiment_TextBlob_Full,sentiment_bert_all,relevant_sent_bert_all
0,8197,8197,VictoriaBC,Relying on rental income in Oak Bay?,We are considering putting an offer on a house...,Relying on rental income in Oak Bay?. We are c...,Neutral,Neutral,Neutral,Neutral,Positive,Positive,Neutral,Negative,Negative
1,1259,1259,VictoriaBC,City Council opposes efforts to increase housi...,"[Link to article: In Fairfield, plans for smal...",City Council opposes efforts to increase housi...,Negative,Negative,Neutral,Neutral,Positive,Positive,Positive,Negative,Negative
2,10403,10403,britishcolumbia,B.C. housing minister tells people not to help...,NaN,B.C. housing minister tells people not to help...,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative
3,10606,10606,britishcolumbia,Vaughn Palmer: End of the line for single-fami...,NaN,Vaughn Palmer: End of the line for single-fami...,Neutral,Neutral,Neutral,Neutral,Positive,Positive,Positive,Positive,Positive
4,618,618,VictoriaBC,Will the housing market ever change for the be...,I’m a part of the younger generation and it se...,Will the housing market ever change for the be...,Negative,Positive,Positive,Neutral,Positive,Positive,Positive,Negative,Negative


In [ ]:
posts =  test_df.TitleText.to_list()

In [ ]:
sentiment_analysis_reddit(posts[0])

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'positive': 0, 'negative': 0, 'neutral': 2}
{'positive': 0, 'negative': 0, 'neutral': 1.5368006825447083}


{'label': 'Neutral', 'score': 0.7684003412723541}

In [ ]:
#Test
sentiment_analysis_reddit("")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'positive': 0, 'negative': 0, 'neutral': 0}
{'positive': 0, 'negative': 0, 'neutral': 0}


{'label': None, 'score': None}

In [ ]:
sentiments = list()
score = list()
cnt = 0
for post in posts:
  print(cnt)
  sentiment = sentiment_analysis_reddit(post)
  cnt+=1
  sentiments.append(sentiment['label'])
  score.append(sentiment['score'])

In [ ]:
test_df["roberta_sentiment"] = sentiments
test_df["roberta_sentiment_score"] = score

In [ ]:
test_df.to_csv("/content/test_set_sentiment_scored.csv")